In [1]:
import os
import sys
import pickle
import csv
import copy
import json
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [3]:
sys.path.append("../../../tfobjdetect/lib")
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

## Loading and preparing label maps

In [4]:
PATH_TO_LABELS = os.path.join('../../../tfobjdetect/lib/object_detection/data', 'oid_bbox_trainable_label_map.pbtxt')
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=600, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [5]:
mid2name = {}
mid2idx = {}
for item in label_map.item:
    mid2name[item.name] = item.display_name
    mid2idx[item.name] = item.id    

In [6]:
with open('../../../tfobjdetect/results/det_results_voa_oi_1.pkl', 'rb') as fin:
    det_results = pickle.load(fin)

## Choosing examples for each class to visualize

In [7]:
label_to_img = {key: [] for key in mid2name}
for key, val in det_results.items():
    for det in val:
        if det['score'] >= 0.1:
            label_to_img[det['label']].append(key)

In [8]:
all_labels = list(label_to_img.keys())

In [9]:
sort_idx = np.argsort([-len(label_to_img[label]) for label in all_labels])

In [10]:
#select_labels = ['/m/01fnck', '/m/019jd', '/m/03120', '/m/02_41', '/m/07cmd', '/m/06q40', '/m/03qtwd', '/m/09x0r', '/m/01xgg_', '/m/08qrwn', '/m/012n4x', '/m/02p16m6', '/m/02lbcq', '/m/09ct_', '/m/0g54v5d', '/m/0bg2p', '/m/0ct4f', '/m/01nd_n', '/m/09rvcxw', '/m/01nl4x', '/m/04ctx', '/m/04ylt', '/m/0gvss07', '/m/01rzcn', '/m/06nrc', '/m/0cyfs', '/m/0f5lx', '/m/04zjc', '/m/0dhz0', '/m/01lcw4', '/m/03htg', '/m/0gxl3', '/m/01bq8v', '/m/0lt4_', '/m/02gzp', '/m/02yjc', '/m/012n7d', '/m/0jb3', '/m/0dwx7']
select_labels = [all_labels[i] for i in sort_idx]

## Visualizing Results

In [11]:
def preproc(im):
    target_size = 400
    max_size = 1024
    im_size_min = np.min(im.shape[0:2])
    im_size_max = np.max(im.shape[0:2])
    im_scale = float(target_size) / float(im_size_min)
    # Prevent the biggest axis from being more than MAX_SIZE
    if np.round(im_scale * im_size_max) > max_size:
        im_scale = float(max_size) / float(im_size_max)
    im = cv2.resize(im, None, None, fx=im_scale, fy=im_scale,
                    interpolation=cv2.INTER_LINEAR)    
    return im

In [12]:
result_dict = {
    'OI': det_results,
}

In [13]:
path_prefix = '../../../../data/rpi_voa/en/'

In [27]:
def show(imgid, source, thresh):
    filename = path_prefix + imgid
    print(imgid)
    with open(filename, 'rb') as fin:
        #_ = fin.read(1024)
        imgbin = fin.read()
    imgbgr = cv2.imdecode(np.frombuffer(imgbin, dtype='uint8'), cv2.IMREAD_COLOR)
    image_np = imgbgr[:,:,[2,1,0]]
    image_np = preproc(image_np)
    
    detections = result_dict[source][imgid]
    boxes = np.asarray([det['bbox_normalized'] for det in detections])[:,[1,0,3,2]]
    scores = [det['score'] for det in detections]
    label_idx = [mid2idx[det['label']] for det in detections]
    
    vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      boxes,
      label_idx,
      scores,
      category_index,
      use_normalized_coordinates=True,
      min_score_thresh=thresh-0.001,
      max_boxes_to_draw=1000,
      line_thickness=2)
    
    plt.figure(figsize=(12, 8))
    plt.imshow(image_np)
    plt.axis('off')
    plt.show()    

In [28]:
%matplotlib inline

In [29]:
for label in select_labels:
    if len(label_to_img[label]) == 0:
        continue
    print(mid2name[label])
    imgid = np.random.choice(label_to_img[label])
    print(imgid)
    w = interactive(show, 
        imgid=fixed(imgid),
        source=widgets.RadioButtons(options=['OI'], value='OI'), 
        thresh=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.1, continuous_update=False), 
    )
    w.children[-1].layout.height = '600px'
    display(w)


Man
VOA_EN_NW_2013.09.26.1757871_0.jpg


interactive(children=(RadioButtons(description='source', options=('OI',), value='OI'), FloatSlider(value=0.1, …

Clothing
VOA_EN_NW_2015.04.30.2742836_0.jpg


interactive(children=(RadioButtons(description='source', options=('OI',), value='OI'), FloatSlider(value=0.1, …

Person
VOA_EN_NW_2016.07.05.3405302_2.jpg


...

interactive(children=(RadioButtons(description='source', options=('OI',), value='OI'), FloatSlider(value=0.1, …

Sports equipment
VOA_EN_NW_2016.08.05.3452132_1.jpg


interactive(children=(RadioButtons(description='source', options=('OI',), value='OI'), FloatSlider(value=0.1, …

Dragonfly
VOA_EN_NW_2017.02.16.3727940_0.jpg


interactive(children=(RadioButtons(description='source', options=('OI',), value='OI'), FloatSlider(value=0.1, …